# 1. Imports

In [1]:
import os
from notebook_init import init_notebook_path

# Get project root path
project_root = init_notebook_path()

# Set environment variable for Ray workers to find the app module
os.environ["PYTHONPATH"] = f"{project_root}:{os.environ.get('PYTHONPATH', '')}"

# Imports
import numpy as np
from app.workflow.utils import read_files_recursive, split_data_granular, smooth_data, merge_close_integers, remove_outliers, filter_extremes, align_min_max_lists_counts_order, add_predicted_transitions_to_df 
from app.utils import calculate_euler_angles_list, unwrap_angles
from app.visualization_utils import draw_plotly_new, plot_predicted_transitions
from app.workflow.gradientdescent import gradient_descent_full




Python path properly configured. Project root added: /Users/kuba/iccs


# 2. Data loading and preprocessing

In [2]:
labeled_data = read_files_recursive(path = os.path.join(project_root, "data", "turns_labeled"))


Processing: /Users/kuba/iccs/data/turns_labeled/HONOR_8X/2024-03-22/6.csv
Processing: /Users/kuba/iccs/data/turns_labeled/HONOR_8X/2024-03-22/7.csv
Processing: /Users/kuba/iccs/data/turns_labeled/HONOR_8X/2024-03-22/5.csv
Processing: /Users/kuba/iccs/data/turns_labeled/HONOR_8X/2024-03-22/4.csv
Processing: /Users/kuba/iccs/data/turns_labeled/HONOR_8X/2024-03-22/1.csv
Processing: /Users/kuba/iccs/data/turns_labeled/HONOR_8X/2024-03-22/3.csv
Processing: /Users/kuba/iccs/data/turns_labeled/HONOR_8X/2024-03-22/2.csv
Processing: /Users/kuba/iccs/data/turns_labeled/HONOR_8X/2024-03-22/10.csv
Processing: /Users/kuba/iccs/data/turns_labeled/HONOR_8X/2024-03-22/9.csv
Processing: /Users/kuba/iccs/data/turns_labeled/HONOR_8X/2024-03-22/8.csv
Processing: /Users/kuba/iccs/data/turns_labeled/HONOR_8X/2024-02-24/2_sp.csv
Processing: /Users/kuba/iccs/data/turns_labeled/HONOR_8X/2024-02-24/6_sp.csv
Processing: /Users/kuba/iccs/data/turns_labeled/HONOR_8X/2024-02-24/4_sp.csv
Processing: /Users/kuba/iccs

#### Lets select single run

In [3]:
input_df = labeled_data[0]
input_df = input_df.drop(columns=['STYLE', 'SKIER_LEVEL', 'SLOPE', "bearingAccuracy_speedAccuracy_verticalAccuracy_horizontalAccuracy_speed_bearing_altitude_longitude_latitude"], errors='ignore')
input_df.head()

,TotalAcceleration,Orientation,Magnetometer,GyroscopeUncalibrated,MagnetometerUncalibrated,Accelerometer,Gyroscope,Gravity,Behavior,Status
time,,,,,,,,,,
2024-03-22 12:49:29.900,"[2.25630521774292, 10.217418670654297, 12.2207...","[-0.6992300152778625, -0.6967610120773315, -0....","[-1.3125, -40.625, 12.8125]","[0.7677528858184814, -0.2957809567451477, -0.2...","[-48.875, -65.25, 48.3125]","[6.934799671173096, -2.1800999641418457, 4.240...","[1.5788772106170654, -0.4668406546115875, -0.5...","[-0.2466999888420105, 9.51550006866455, 2.3587...",right,START
2024-03-22 12:49:30.000,"[0.2472539991140365, 7.208529472351074, 6.4393...","[-0.7047449946403503, -0.681315004825592, -0.1...","[-2.875, -41.5625, 9.125]","[1.3339725732803345, 0.1136732921004295, 0.069...","[-50.4375, -66.1875, 44.625]","[0.6118999719619751, -2.504899978637696, -2.54...","[0.6350031495094299, -0.1096939444541931, -0.2...","[-0.0528999976813793, 9.330399513244627, 3.018...",,
2024-03-22 12:49:30.100,"[2.0676231384277344, 8.164840698242188, 3.4410...","[-0.7018659710884094, -0.6656039953231812, -0....","[-3.25, -42.625, 4.1875]","[0.9653939604759216, -0.339204728603363, -0.49...","[-50.8125, -67.25, 39.6875]","[1.2264000177383425, 0.9109999537467957, 2.181...","[1.4157238006591797, -0.1066570729017257, 0.07...","[-0.0495999976992607, 8.8464994430542, 4.23149...",,
2024-03-22 12:49:30.200,"[3.169674158096313, 7.666990280151367, 6.17287...","[-0.7076629996299744, -0.6421110033988953, -0....","[-5.0625, -42.5625, -0.3125]","[0.965795397758484, -1.0057809352874756, -0.69...","[-52.625, -67.1875, 35.1875]","[-2.747900009155273, -1.5245999097824097, 0.94...","[0.6662619709968567, -0.6124709248542786, -0.6...","[0.1098999977111816, 8.487299919128418, 4.9113...",,
2024-03-22 12:49:30.300,"[-0.5638249516487122, 6.842130184173584, 1.203...","[-0.7049899697303772, -0.6053839921951294, -0....","[-8.125, -41.5625, -4.5625]","[1.132945537567139, -1.967439889907837, 0.0192...","[-55.6875, -66.1875, 30.9375]","[-0.8007000088691711, -4.292599678039551, -1.0...","[1.258434772491455, -1.073429822921753, -0.755...","[0.2220999896526336, 7.881799697875977, 5.8306...",,


#### Get just orientation data

In [4]:
split_on = "Orientation"
orientation_df = split_data_granular([input_df], split_on=split_on)[0]
display(orientation_df.head())
draw_plotly_new(orientation_df, col=split_on, granular=True, title="Orientation Data Visualization")

,qz,qy,qx,qw,roll,pitch,yaw,Behavior,Status
time,,,,,,,,,
2024-03-22 12:49:29.900,-0.699230,-0.696761,-0.157973,0.025403,-1.652011,-1.310705,1.678938,right,START
2024-03-22 12:49:30.000,-0.704745,-0.681315,-0.193328,0.042032,-1.580293,-1.234717,1.778738,,
2024-03-22 12:49:30.100,-0.701866,-0.665604,-0.236985,0.090509,-1.567224,-1.100493,1.787777,,
2024-03-22 12:49:30.200,-0.707663,-0.642111,-0.278845,0.095662,-1.532405,-1.026413,1.878965,,
2024-03-22 12:49:30.300,-0.704990,-0.605384,-0.358699,0.088502,-1.554987,-0.910953,1.993297,,


#### Calculate Euler angles based on quaternions

In [5]:
transformed_df = calculate_euler_angles_list([orientation_df])[0]
display(transformed_df.head())
draw_plotly_new(transformed_df, col=split_on, granular=True, title="Transformed Orientation Data Visualization")

,qz,qy,qx,qw,roll,pitch,yaw,Behavior,Status
time,,,,,,,,,
2024-03-22 12:49:29.900,-0.699230,-0.696761,-0.157973,0.025403,1.592382,-0.259212,2.949417,right,START
2024-03-22 12:49:30.000,-0.704745,-0.681315,-0.193328,0.042032,1.574114,-0.336058,2.923579,,
2024-03-22 12:49:30.100,-0.701866,-0.665604,-0.236985,0.090509,1.568980,-0.470295,2.928611,,
2024-03-22 12:49:30.200,-0.707663,-0.642111,-0.278845,0.095662,1.547564,-0.543936,2.878292,,
2024-03-22 12:49:30.300,-0.704990,-0.605384,-0.358699,0.088502,1.558532,-0.659743,2.739093,,


#### Unwrap the data

In [6]:
unwrapped_data = unwrap_angles([transformed_df])[0]
display(unwrapped_data.head())
draw_plotly_new(unwrapped_data, col=split_on, granular=True, title="Unwrapped Orientation Data Visualization")


,qz,qy,qx,qw,roll,pitch,yaw,Behavior,Status
time,,,,,,,,,
2024-03-22 12:49:29.900,-0.699230,-0.696761,-0.157973,0.025403,1.592382,-0.259212,2.949417,right,START
2024-03-22 12:49:30.000,-0.704745,-0.681315,-0.193328,0.042032,1.574114,-0.336058,2.923579,,
2024-03-22 12:49:30.100,-0.701866,-0.665604,-0.236985,0.090509,1.568980,-0.470295,2.928611,,
2024-03-22 12:49:30.200,-0.707663,-0.642111,-0.278845,0.095662,1.547564,-0.543936,2.878292,,
2024-03-22 12:49:30.300,-0.704990,-0.605384,-0.358699,0.088502,1.558532,-0.659743,2.739093,,


#### Smooth the data



In [7]:
smoothed_orientation_df = smooth_data([unwrapped_data], window_size=10)[0]
display(smoothed_orientation_df.head())
draw_plotly_new(smoothed_orientation_df, col=split_on, granular=True, title="Smoothed Orientation Data Visualization")

,qz,qy,qx,qw,roll,pitch,yaw,Behavior,Status
time,,,,,,,,,
2024-03-22 12:49:29.900,-0.699230,-0.696761,-0.157973,0.025403,1.592382,-0.259212,2.949417,right,START
2024-03-22 12:49:30.000,-0.704745,-0.681315,-0.193328,0.042032,1.574114,-0.336058,2.923579,,
2024-03-22 12:49:30.100,-0.701866,-0.665604,-0.236985,0.090509,1.568980,-0.470295,2.928611,,
2024-03-22 12:49:30.200,-0.707663,-0.642111,-0.278845,0.095662,1.547564,-0.543936,2.878292,,
2024-03-22 12:49:30.300,-0.704990,-0.605384,-0.358699,0.088502,1.558532,-0.659743,2.739093,,


#### Remove labels

In [8]:
raw_orientation_df = smoothed_orientation_df.drop(columns=['Behavior', 'Status'], errors='ignore')
display(raw_orientation_df.head())
draw_plotly_new(raw_orientation_df, col=split_on, granular=True, title="Raw Orientation Data Visualization")

,qz,qy,qx,qw,roll,pitch,yaw
time,,,,,,,
2024-03-22 12:49:29.900,-0.699230,-0.696761,-0.157973,0.025403,1.592382,-0.259212,2.949417
2024-03-22 12:49:30.000,-0.704745,-0.681315,-0.193328,0.042032,1.574114,-0.336058,2.923579
2024-03-22 12:49:30.100,-0.701866,-0.665604,-0.236985,0.090509,1.568980,-0.470295,2.928611
2024-03-22 12:49:30.200,-0.707663,-0.642111,-0.278845,0.095662,1.547564,-0.543936,2.878292
2024-03-22 12:49:30.300,-0.704990,-0.605384,-0.358699,0.088502,1.558532,-0.659743,2.739093


# 3. Forecast Turns 

In [9]:
# !For now just copy entire section - later will ve extracted into something more readable

df = raw_orientation_df.copy()
col = "yaw"

start_indices = np.random.randint(1, len(df), size=200)

predicted_mins, predicted_maxs = gradient_descent_full(df, start_indices, learning_rate=0.054, steps=300, momentum=0.950, selected_col= col, printing=False)

unique_predictions_mins, counts_mins = np.unique(predicted_mins, return_counts=True)
unique_predictions_maxs, counts_maxs = np.unique(predicted_maxs, return_counts=True)

new_mins, new_counts_min = merge_close_integers(unique_predictions_mins, counts_mins, threshold = 13)
new_max, new_counts_max = merge_close_integers(unique_predictions_maxs, counts_maxs, threshold = 13)

new_filtered_mins, new_filtered_mins_counts = remove_outliers(new_mins, new_counts_min, df, multiplier= 3.393, selected_col=col, boundary="upper")
new_filtered_max, new_filtered_maxs_counts = remove_outliers(new_max, new_counts_max, df, 3.393, selected_col=col, boundary="lower")

important_min, important_min_counts = filter_extremes(new_filtered_mins, new_filtered_mins_counts,threshold_int=0)
important_max, important_max_counts = filter_extremes(new_filtered_max, new_filtered_maxs_counts,threshold_int=0)

new_method_mins, new_method_max = align_min_max_lists_counts_order(important_min, important_max, important_min_counts, important_max_counts)
interval_df = add_predicted_transitions_to_df(df, new_method_mins, new_method_max)


In [10]:
display(interval_df.head())
plot_predicted_transitions(interval_df)


,qz,qy,qx,qw,roll,pitch,yaw,Predicted
time,,,,,,,,
2024-03-22 12:49:29.900,-0.699230,-0.696761,-0.157973,0.025403,1.592382,-0.259212,2.949417,right
2024-03-22 12:49:30.000,-0.704745,-0.681315,-0.193328,0.042032,1.574114,-0.336058,2.923579,right
2024-03-22 12:49:30.100,-0.701866,-0.665604,-0.236985,0.090509,1.568980,-0.470295,2.928611,right
2024-03-22 12:49:30.200,-0.707663,-0.642111,-0.278845,0.095662,1.547564,-0.543936,2.878292,right
2024-03-22 12:49:30.300,-0.704990,-0.605384,-0.358699,0.088502,1.558532,-0.659743,2.739093,right


In [11]:
# Add 'Predicted' column from interval_df to input_df based on index
input_df['Predicted'] = interval_df['Predicted']

# Remove 'Behavior' and 'Status' columns if they exist
input_df = input_df.drop(columns=['Behavior', 'Status'], errors='ignore')

# Display the updated input_df
display(input_df)

,TotalAcceleration,Orientation,Magnetometer,GyroscopeUncalibrated,MagnetometerUncalibrated,Accelerometer,Gyroscope,Gravity,Predicted
time,,,,,,,,,
2024-03-22 12:49:29.900,"[2.25630521774292, 10.217418670654297, 12.2207...","[-0.6992300152778625, -0.6967610120773315, -0....","[-1.3125, -40.625, 12.8125]","[0.7677528858184814, -0.2957809567451477, -0.2...","[-48.875, -65.25, 48.3125]","[6.934799671173096, -2.1800999641418457, 4.240...","[1.5788772106170654, -0.4668406546115875, -0.5...","[-0.2466999888420105, 9.51550006866455, 2.3587...",right
2024-03-22 12:49:30.000,"[0.2472539991140365, 7.208529472351074, 6.4393...","[-0.7047449946403503, -0.681315004825592, -0.1...","[-2.875, -41.5625, 9.125]","[1.3339725732803345, 0.1136732921004295, 0.069...","[-50.4375, -66.1875, 44.625]","[0.6118999719619751, -2.504899978637696, -2.54...","[0.6350031495094299, -0.1096939444541931, -0.2...","[-0.0528999976813793, 9.330399513244627, 3.018...",right
2024-03-22 12:49:30.100,"[2.0676231384277344, 8.164840698242188, 3.4410...","[-0.7018659710884094, -0.6656039953231812, -0....","[-3.25, -42.625, 4.1875]","[0.9653939604759216, -0.339204728603363, -0.49...","[-50.8125, -67.25, 39.6875]","[1.2264000177383425, 0.9109999537467957, 2.181...","[1.4157238006591797, -0.1066570729017257, 0.07...","[-0.0495999976992607, 8.8464994430542, 4.23149...",right
2024-03-22 12:49:30.200,"[3.169674158096313, 7.666990280151367, 6.17287...","[-0.7076629996299744, -0.6421110033988953, -0....","[-5.0625, -42.5625, -0.3125]","[0.965795397758484, -1.0057809352874756, -0.69...","[-52.625, -67.1875, 35.1875]","[-2.747900009155273, -1.5245999097824097, 0.94...","[0.6662619709968567, -0.6124709248542786, -0.6...","[0.1098999977111816, 8.487299919128418, 4.9113...",right
2024-03-22 12:49:30.300,"[-0.5638249516487122, 6.842130184173584, 1.203...","[-0.7049899697303772, -0.6053839921951294, -0....","[-8.125, -41.5625, -4.5625]","[1.132945537567139, -1.967439889907837, 0.0192...","[-55.6875, -66.1875, 30.9375]","[-0.8007000088691711, -4.292599678039551, -1.0...","[1.258434772491455, -1.073429822921753, -0.755...","[0.2220999896526336, 7.881799697875977, 5.8306...",right
...,...,...,...,...,...,...,...,...,...
2024-03-22 12:49:54.400,"[-3.259428024291992, 3.4100520610809326, 1.163...","[0.3122060000896454, -0.275068998336792, 0.485...","[-29.875, -18.9375, -25.1875]","[-0.2570695579051971, -1.4800392389297483, -0....","[-80.375, -42.0, 9.375]","[4.305900096893311, 11.089599609375, -3.883599...","[-0.3685437142848968, -0.1895951181650161, -1....","[3.647199869155884, 5.709099769592285, 7.09039...",right
2024-03-22 12:49:54.500,"[8.356243133544922, 21.290218353271484, 7.6015...","[0.2947669923305511, -0.2936370074748993, 0.48...","[-30.25, -17.1875, -26.5]","[-0.3126408159732818, 0.0855734944343566, 2.43...","[-80.75, -40.25, 8.0625]","[9.801499366760254, -3.584299802780152, -2.332...","[0.0719948336482048, -0.2573662400245666, -1.2...","[3.6726999282836914, 5.506399631500244, 7.2359...",right
2024-03-22 12:49:54.600,"[16.849567413330078, 20.96907997131348, 17.360...","[0.3042170107364654, -0.2868239879608154, 0.49...","[-30.5, -17.375, -27.25]","[-0.0739495977759361, 0.4781503975391388, -0.0...","[-81.0, -40.4375, 7.3125]","[-2.7049999237060547, 0.4462999999523163, -5.4...","[0.0656243786215782, -0.2276083827018737, -0.3...","[3.3227999210357666, 5.65339994430542, 7.29159...",right
